# ETL_EDA
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. 

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import logging
import utils
import sched
import time
import pymongo

## 1. Raw Data from NASA 
The [dataset](https://api.nasa.gov/assets/insight/InSight%20Weather%20API%20Documentation.pdf) is a continuously updated json file. It contains data of Mars weather in the past seven days. It can be retrieved simply by making `requests` without parameters. Data is updated every Martian day, which is 37 minutes longer than earth day. 

In [2]:
key = "j8jSUUXusA5glsJa3q5bQaDad6sE4H2u9K5rFbST"
requested = requests.get(f"https://api.nasa.gov/insight_weather/?api_key={key}&feedtype=json&ver=1.0").text    # Download data
requested

'{\n  "366": {\n    "AT": {\n      "av": -61.513, \n      "ct": 248110, \n      "mn": -98.941, \n      "mx": -20.442\n    }, \n    "First_UTC": "2019-12-07T06:39:48Z", \n    "HWS": {\n      "av": 5.6, \n      "ct": 119607, \n      "mn": 0.242, \n      "mx": 20.351\n    }, \n    "Last_UTC": "2019-12-08T00:34:43Z", \n    "PRE": {\n      "av": 660.866, \n      "ct": 123645, \n      "mn": 646.4396, \n      "mx": 674.6912\n    }, \n    "Season": "spring", \n    "WD": {\n      "10": {\n        "compass_degrees": 225.0, \n        "compass_point": "SW", \n        "compass_right": -0.707106781187, \n        "compass_up": -0.707106781187, \n        "ct": 16069\n      }, \n      "11": {\n        "compass_degrees": 247.5, \n        "compass_point": "WSW", \n        "compass_right": -0.923879532511, \n        "compass_up": -0.382683432365, \n        "ct": 1981\n      }, \n      "12": {\n        "compass_degrees": 270.0, \n        "compass_point": "W", \n        "compass_right": -1.0, \n        "com

## 2. Raw Data to Documents/Dicts
Using json and pandas package, we can easily transform downloaded json files into workable dataframe. We take sol days as keys, extract min, max temperature, atmosphere pressure and wind directions as values for each day. But not all data is valid. To deal with this, we check the validity value of each subject for each day, then replace missing values with last day's corresponding value. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [3]:
requested = json.loads(requested)    # Transform data into dictionary
main_keys = requested['sol_keys']
def valid_check(subject, index):
    """Check if value is valid"""
    return requested['validity_checks'][index][subject]['valid']


df_requested = {"sol_day": [-1], "date": [-1], "min_temp": [-1], "max_temp": [-1], "pressure": [-1], "wind": [-1]}

for i in main_keys:
    df_requested["sol_day"].append(i)
    df_requested["date"].append(requested[i]['Last_UTC'])
    df_requested["min_temp"].append(requested[i]['AT']["mn"] if valid_check('AT', i) else df_requested["min_temp"][-1])
    df_requested["max_temp"].append(requested[i]['AT']["mx"] if valid_check('AT', i) else df_requested["max_temp"][-1])
    df_requested["pressure"].append(requested[i]['PRE']["av"] if valid_check('PRE', i) else df_requested["pressure"][-1])
    df_requested["wind"].append(requested[i]["WD"] if valid_check('WD', i) else df_requested["wind"][-1])

df_requested = pd.DataFrame(df_requested)
df_requested.drop(0, inplace=True)
data = df_requested.to_dict(orient='records')

In [4]:
df_requested.head()

,sol_day,date,min_temp,max_temp,pressure,wind
1,366,2019-12-08T00:34:43Z,-98.941,-20.442,660.866,"{'10': {'compass_degrees': 225.0, 'compass_poi..."
2,367,2019-12-09T07:58:59Z,-139.936,-20.914,661.043,"{'1': {'compass_degrees': 22.5, 'compass_point..."
3,368,2019-12-10T08:38:35Z,-100.049,-20.919,660.436,"{'1': {'compass_degrees': 22.5, 'compass_point..."
4,369,2019-12-11T09:18:10Z,-98.234,-21.164,659.424,"{'1': {'compass_degrees': 22.5, 'compass_point..."
5,370,2019-12-12T09:57:46Z,-98.970,-19.470,658.193,"{'0': {'compass_degrees': 0.0, 'compass_point'..."


In [5]:
data

[{'sol_day': '366',
  'date': '2019-12-08T00:34:43Z',
  'min_temp': -98.941,
  'max_temp': -20.442,
  'pressure': 660.866,
  'wind': {'10': {'compass_degrees': 225.0,
    'compass_point': 'SW',
    'compass_right': -0.707106781187,
    'compass_up': -0.707106781187,
    'ct': 16069},
   '11': {'compass_degrees': 247.5,
    'compass_point': 'WSW',
    'compass_right': -0.923879532511,
    'compass_up': -0.382683432365,
    'ct': 1981},
   '12': {'compass_degrees': 270.0,
    'compass_point': 'W',
    'compass_right': -1.0,
    'compass_up': -0.0,
    'ct': 441},
   '13': {'compass_degrees': 292.5,
    'compass_point': 'WNW',
    'compass_right': -0.923879532511,
    'compass_up': 0.382683432365,
    'ct': 48},
   '15': {'compass_degrees': 337.5,
    'compass_point': 'NNW',
    'compass_right': -0.382683432365,
    'compass_up': 0.923879532511,
    'ct': 2},
   '3': {'compass_degrees': 67.5,
    'compass_point': 'ENE',
    'compass_right': 0.923879532511,
    'compass_up': 0.382683432365

## 3. Upsert MongoDB
If we fetch the data frequently, there are lots of duplicate data entry between each run. The de-duplication happens at insertion. The MongoDB API to use is `collection.replace_one(filter=..., replacement=..., upsert=True)`. The statement matches a document in MongoDB with `filter`, replaces it with `replacement` if the document exists or inserts `replacement` into the database if `filter` matches nothing. Credit to [Martin Ma](https://github.com/blownhither). 

In [6]:
client = pymongo.MongoClient('localhost', 27017)
db = client.get_database("MarsWeather")
collection = db.get_collection("DailyWeather")
update_count = 0
for record in data:
    result = collection.replace_one(
        filter={'sol_day': record['sol_day']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df_requested.shape[0]}, update={update_count}, "
      f"insert={df_requested.shape[0]-update_count}")

rows=7, update=7, insert=0
